In [ ]:
import os
from tqdm import tqdm
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from open3d.web_visualizer import draw


In [2]:
from quickstart.data import point_cloud_data
from quickstart.code.preprocess.data_gen import crop_pc_z, get_density


# Load point cloud data

In [3]:
filename = "S_29_0001_2_I"
pcd_path = f"quickstart/data/aihub/{filename}.ply"
pcd = point_cloud_data.read_point_cloud(pcd_path)  # PointCloud with 1,941,471 points.


In [ ]:
# also meters to feet  :1397
pcd.points = o3d.utility.Vector3dVector(np.array(pcd.points) / 0.3048)
np.asarray(pcd.points).shape  # (1941471, 3)


In [ ]:
bounds_min = pcd.get_min_bound()
bounds_max = pcd.get_max_bound()

print(f"bounds_min: {bounds_min}")
print(f"bounds_max: {bounds_max}")


In [6]:
# determine point cloud bounds
bounds_minx, bounds_miny, _ = bounds_min * 12 - 15
bounds_maxx, bounds_maxy, _ = bounds_max * 12 + 15

h = round(bounds_maxy - bounds_miny)
w = round(bounds_maxx - bounds_minx)

# Cropping point cloud according to bound
bounds = np.array([bounds_minx, bounds_miny, bounds_maxx, bounds_maxy])

bounds_ft = bounds / 12


# Crop


In [ ]:
from quickstart.code.preprocess.data_gen import crop_pc

cropped_pcd = crop_pc(pcd, bounds_ft)
np.asarray(cropped_pcd.points).shape  # (1941471, 3)


In [ ]:
# 크롭된 포인트 클라우드 시각화
draw([cropped_pcd])


# Slicing

## Slicing intervals

In [ ]:
min_z = bounds_min[-1]
max_z = bounds_max[-1]
slice_intervals = np.linspace(min_z, max_z, 8)


In [10]:
center_slice_points = []
for cz in slice_intervals:
    cx, cy, _ = (bounds_min + bounds_max) / 2
    center_slice_points.append([cx, cy, cz])
centers_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(center_slice_points))


In [ ]:
centers_pcd.colors = o3d.utility.Vector3dVector(
    np.array([[1, 0, 0] for _ in range(len(center_slice_points))])
)
draw([cropped_pcd, centers_pcd])


## Slice

In [ ]:
_length = (pcd.get_max_bound() * 12 - 15) - (pcd.get_min_bound() * 12 + 15)
length = _length.astype(int)
width, height, _ = length


In [ ]:
density_slices = []
for slice_i, (slice_minz, slice_maxz) in enumerate(
    tqdm(list(zip(slice_intervals[:-1], slice_intervals[1:])))
):
    sliced_pcd = crop_pc_z(cropped_pcd, slice_minz, slice_maxz)  # (700200, 3)
    density_slice = get_density(
        sliced_pcd,  # (700200, 3)
        bounds_ft,  # [-1, -1, 26, 17]
        width,  # 269
        height,  # 160
    )
    density_slices.append(density_slice)


## Caching density slice

In [16]:
if not os.path.exists(f"quickstart/data/density/{filename}"):
    os.makedirs(f"quickstart/data/density/{filename}")
    for i in range(7):
        npy_path = f"quickstart/data/density/{filename}/density_{i:02d}.npy"
        np.save(npy_path, density_slices[i])


## Visualize density slice

In [17]:
density_slices = []
for i in range(7):
    npy_path = f"quickstart/data/density/{filename}/density_{i:02d}.npy"
    density_map = np.load(npy_path)
    density_slices.append(density_map)


In [ ]:
num_plot = len(density_slices)  # 7
fig, axs = plt.subplots(3, 3, figsize=(15, 10))
for i in range(9):  # 0..6
    ax = axs[i // 3, i % 3]
    if i >= num_plot:
        ax.set_visible(False)
        ax.axis("off")
    else:
        ax.imshow(density_slices[i])
plt.show()
